In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision.models import resnet18
import torch.nn.functional as F  
import torchvision.models as models  # Import this line to access pre-trained models
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc

In [4]:
def merge_csv_files(directory, output_file, prefix="train_segment_merged_part_"):
    # Identify files with the specified prefix and .csv extension
    csv_files = [os.path.join(directory, f) for f in os.listdir(directory) 
                 if f.startswith(prefix) and f.endswith('.csv')]
    
    # Load and concatenate all identified files
    df_list = []
    for file in csv_files:
        print(f"Reading file: {file}")  # Print the name of each file being read
        df_list.append(pd.read_csv(file))
    
    merged_df = pd.concat(df_list, ignore_index=True)
    
    # Save the merged dataframe to a single CSV file
    merged_df.to_csv(output_file, index=False)
    return merged_df


In [12]:
def preprocess_data(df):
    # Extract features and labels
    file_names = df.iloc[:, 0]  # File names (not used for training)
    labels = df.iloc[:, -1].values  # Labels (1=real, 0=fake)
    features = df.iloc[:, 1:-1].values  # Features (1x1084)
    
    # Normalize features
    scaler = StandardScaler()
    features_normalized = scaler.fit_transform(features)
    
    # Split data
    X_train, X_temp, y_train, y_temp = train_test_split(features_normalized, labels, test_size=0.4, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
    
    return X_train, X_val, X_test, y_train, y_val, y_test


In [14]:
class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)


In [15]:
class AttentionModel(torch.nn.Module):
    def __init__(self, input_dim, embedding_dim, num_classes):
        super(AttentionModel, self).__init__()
        self.fc = torch.nn.Linear(input_dim, embedding_dim)
        self.attn = torch.nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=2)
        self.classifier = torch.nn.Linear(embedding_dim, num_classes)

    def forward(self, x):
        x = self.fc(x)
        attn_output, _ = self.attn(x.unsqueeze(0), x.unsqueeze(0), x.unsqueeze(0))  # Apply attention mechanism
        x = attn_output.squeeze(0)
        logits = self.classifier(x)
        return logits, x  # Return both logits and embeddings


In [29]:
def train_contrastive_model(model, criterion, optimizer, train_loader, val_loader=None, epochs=50):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    best_auc = 0
    best_eer = 1
    best_model = None

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()

            # Forward pass
            logits, _ = model(x)

            # Compute loss
            loss = nn.CrossEntropyLoss()(logits, y)  # Using cross-entropy for classification
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

        if val_loader:
            # Evaluate model on validation set after each epoch
            auc_score, cm, eer = test_model(model, val_loader, device)
            print(f"Validation AUC: {auc_score:.4f}, EER: {eer:.4f}")

            # Store the best model
            if eer < best_eer or auc_score > best_auc:
                best_auc = auc_score
                best_eer = eer
                best_model = model.state_dict()

    # Save the best model
    if best_model:
        model.load_state_dict(best_model)
        print(f"Best model saved with AUC: {best_auc:.4f}, EER: {best_eer:.4f}")


In [25]:
def calculate_eer(fpr, tpr):
    # EER is the point where FPR and (1 - TPR) are closest to each other
    fnr = 1 - tpr
    eer_idx = np.nanargmin(np.abs(fpr - fnr))  # Find the index where the difference is smallest
    eer = fpr[eer_idx]  # The value of FPR at that index is the EER
    return eer


In [27]:
def test_model(model, val_loader, device="cuda"):
    model.eval()  # Set the model to evaluation mode
    all_labels = []
    all_preds = []

    with torch.no_grad():  # No gradients needed for evaluation
        for data, labels in val_loader:
            data, labels = data.to(device), labels.to(device)
            logits, _ = model(data)
            preds = torch.softmax(logits, dim=1)[:, 1]  # Get probabilities for the positive class
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    all_labels = np.array(all_labels)
    all_preds = np.array(all_preds)

    # Calculate AUC
    auc_score = roc_auc_score(all_labels, all_preds)

    # Confusion Matrix
    cm = confusion_matrix(all_labels, (all_preds > 0.5).astype(int))

    # Compute ROC curve and EER
    fpr, tpr, thresholds = roc_curve(all_labels, all_preds)
    eer = calculate_eer(fpr, tpr)

    return auc_score, cm, eer  # Return all values needed


In [8]:
def plot_tsne(features, labels, title, save_path):
    tsne = TSNE(n_components=2, random_state=42)
    tsne_features = tsne.fit_transform(features)
    plt.figure(figsize=(8, 8))
    plt.scatter(tsne_features[:, 0], tsne_features[:, 1], c=labels, cmap='viridis', s=10)
    plt.colorbar()
    plt.title(title)
    plt.savefig(save_path)
    plt.show()


In [ ]:
def save_model(model, save_path):
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to {save_path}")


In [9]:
def extract_embeddings(model, features):
    model.eval()
    with torch.no_grad():
        embeddings, _ = model(torch.tensor(features, dtype=torch.float32).cuda())
    return embeddings.cpu().numpy()

def save_embeddings_to_csv(file_ids, embeddings, labels, output_file):
    embeddings_df = pd.DataFrame({
        "file_id": file_ids,
        "embedding": [emb.tolist() for emb in embeddings],
        "label": labels
    })
    embeddings_df.to_csv(output_file, index=False)
    print(f"Embeddings saved to {output_file}")


In [16]:
def load_model_from_local(model_path, weights_path, device="cuda"):
    device = torch.device(device if torch.cuda.is_available() else "cpu")
    
    # Load the full model
    loaded_model = torch.load(model_path, map_location=device)
    loaded_model.eval()  # Set the model to evaluation mode
    print("Model and weights successfully loaded from local directory.")
    return loaded_model


In [17]:
# 1. Define paths and files
csv_dir = r"C:\Notebooks\rrl_source\dataset_raw\merge\segmented_features\train\\"  # Update with your directory path
merged_output_file = "merged_data.csv"
save_dir = r"C:\Notebooks\rrl_source\dataset_raw\merge\segmented_features\train\\"  # Update with your desired save directory

# 2. Merge the CSV files
merged_data = merge_csv_files(csv_dir, merged_output_file)

# 3. Preprocess data
X_train, X_val, X_test, y_train, y_val, y_test = preprocess_data(merged_data)

Reading file: C:\Notebooks\rrl_source\dataset_raw\merge\segmented_features\train\\train_segment_merged_part_1.csv
Reading file: C:\Notebooks\rrl_source\dataset_raw\merge\segmented_features\train\\train_segment_merged_part_2.csv
Reading file: C:\Notebooks\rrl_source\dataset_raw\merge\segmented_features\train\\train_segment_merged_part_3.csv
Reading file: C:\Notebooks\rrl_source\dataset_raw\merge\segmented_features\train\\train_segment_merged_part_4.csv
Reading file: C:\Notebooks\rrl_source\dataset_raw\merge\segmented_features\train\\train_segment_merged_part_5.csv
Reading file: C:\Notebooks\rrl_source\dataset_raw\merge\segmented_features\train\\train_segment_merged_part_6.csv
Reading file: C:\Notebooks\rrl_source\dataset_raw\merge\segmented_features\train\\train_segment_merged_part_7.csv


In [18]:
# 4. Create DataLoader
train_dataset = AudioDataset(X_train, y_train)
val_dataset = AudioDataset(X_val, y_val)
test_dataset = AudioDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [31]:
# 5. Initialize the model
input_dim = X_train.shape[1]
embedding_dim = 128
num_classes = 2
model = AttentionModel(input_dim=input_dim, embedding_dim=embedding_dim, num_classes=num_classes).cuda()

# 6. Define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
criterion = torch.nn.CrossEntropyLoss()

# 7. Train the model
train_contrastive_model(model, criterion, optimizer, train_loader, val_loader, epochs=100)


Epoch 1/100, Loss: 0.4960
Validation AUC: 0.8653, EER: 0.2119
Best Model Saved with EER: 0.2119326241134752, AUC: 0.865327665627423
Epoch 2/100, Loss: 0.4615
Validation AUC: 0.8680, EER: 0.2102
Best Model Saved with EER: 0.2102127659574468, AUC: 0.8680128821642937
Epoch 3/100, Loss: 0.4539
Validation AUC: 0.8791, EER: 0.1983
Best Model Saved with EER: 0.19831560283687943, AUC: 0.8790616604350189
Epoch 4/100, Loss: 0.4488
Validation AUC: 0.8849, EER: 0.1932
Best Model Saved with EER: 0.19319148936170213, AUC: 0.8849224967499799
Epoch 5/100, Loss: 0.4459
Validation AUC: 0.8780, EER: 0.1997
Best Model Saved with EER: 0.19319148936170213, AUC: 0.8849224967499799
Epoch 6/100, Loss: 0.4433
Validation AUC: 0.8707, EER: 0.2049
Best Model Saved with EER: 0.19319148936170213, AUC: 0.8849224967499799
Epoch 7/100, Loss: 0.4412
Validation AUC: 0.8677, EER: 0.2100
Best Model Saved with EER: 0.19319148936170213, AUC: 0.8849224967499799
Epoch 8/100, Loss: 0.4389
Validation AUC: 0.8854, EER: 0.1923
Bes

In [30]:
pwd

'C:\\Notebooks\\rrl_source\\Spectnet_model_embedding'

In [32]:
import os
import torch

# Define the save directory (absolute path)
save_dir = r"C:\Notebooks\rrl_source\dataset_raw\merge\segmented_features\train"

# 1. Ensure the save directory exists
os.makedirs(save_dir, exist_ok=True)

# 2. Define the path to save the model and weights
model_path = os.path.join(save_dir, "trained_model.pth")
weights_path = os.path.join(save_dir, "model_weights.pth")

# 3. Save the entire model (architecture + weights)
torch.save(model, model_path)  # Saves both the model architecture and weights

# 4. Save only the model weights (state_dict)
torch.save(model.state_dict(), weights_path)

print(f"Model saved to {model_path}")
print(f"Model weights saved to {weights_path}")



Model saved to C:\Notebooks\rrl_source\dataset_raw\merge\segmented_features\train\trained_model.pth
Model weights saved to C:\Notebooks\rrl_source\dataset_raw\merge\segmented_features\train\model_weights.pth


In [ ]:
# Load the entire model (architecture + weights)
model = torch.load(model_path)
model.eval()  # Set the model to evaluation mode for inference


In [ ]:

# 9. Test the model
test_model(model, test_loader)

